In [183]:
import tensorflow as tf

from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import RandomNormal, Zeros

In [184]:
class SequencePoolingLayer(Layer):
    """The SequencePoolingLayer is used to apply pooling operation(sum,mean,max) on variable-length sequence feature/multi-value feature.

      Input shape
        - A list of two  tensor [seq_value,seq_len]

        - seq_value is a 3D tensor with shape: ``(batch_size, T, embedding_size)``

        - seq_len is a 2D tensor with shape : ``(batch_size, 1)``,indicate valid length of each sequence.

      Output shape
        - 3D tensor with shape: ``(batch_size, 1, embedding_size)``.

      Arguments
        - **mode**:str.Pooling operation to be used,can be sum,mean or max.

        - **supports_masking**:If True,the input need to support masking.
    """

    def __init__(self, mode='mean', supports_masking=False, **kwargs):

        if mode not in ['sum', 'mean', 'max']:
            raise ValueError("mode must be sum or mean")
        self.mode = mode
        self.eps = tf.constant(1e-8, tf.float32)
        super(SequencePoolingLayer, self).__init__(**kwargs)

        self.supports_masking = supports_masking

    def build(self, input_shape):
        if not self.supports_masking:
            self.seq_len_max = int(input_shape[0][1])
        super(SequencePoolingLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, seq_value_len_list, mask=None, **kwargs):
        if self.supports_masking:
            if mask is None:
                raise ValueError(
                    "When supports_masking=True,input must support masking")
            uiseq_embed_list = seq_value_len_list
            mask = tf.cast(mask, tf.float32)  # tf.to_float(mask)
            user_behavior_length = tf.reduce_sum(mask, axis=-1, keepdims=True)
            mask = tf.expand_dims(mask, axis=2)
        else:
            uiseq_embed_list, user_behavior_length = seq_value_len_list

            mask = tf.sequence_mask(user_behavior_length,
                                    self.seq_len_max, dtype=tf.float32)
            mask = tf.transpose(mask, (0, 2, 1))

        embedding_size = uiseq_embed_list.shape[-1]

        mask = tf.tile(mask, [1, 1, embedding_size])

        if self.mode == "max":
            hist = uiseq_embed_list - (1-mask) * 1e9
            return tf.reduce_max(hist, 1, keepdims=True)

        hist = tf.reduce_sum(uiseq_embed_list * mask, 1, keepdims=False)

        if self.mode == "mean":
            hist = tf.divide(hist, tf.cast(user_behavior_length, tf.float32) + self.eps)

        hist = tf.expand_dims(hist, axis=1)
        return hist

    def compute_output_shape(self, input_shape):
        if self.supports_masking:
            return (None, 1, input_shape[-1])
        else:
            return (None, 1, input_shape[0][-1])

    def compute_mask(self, inputs, mask):
        return None

    def get_config(self, ):
        config = {'mode': self.mode, 'supports_masking': self.supports_masking}
        base_config = super(SequencePoolingLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [185]:
class LabelAwareAttention(Layer):
    def __init__(self, k_max, pow_p=1, **kwargs):
        self.k_max = k_max
        self.pow_p = pow_p
        super(LabelAwareAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        # Be sure to call this somewhere!
        self.embedding_size = input_shape[0][-1]
        super(LabelAwareAttention, self).build(input_shape)

    def call(self, inputs, training=None, **kwargs):
        keys = inputs[0]
        query = inputs[1]
        weight = tf.reduce_sum(keys * query, axis=-1, keepdims=True)
        weight = tf.pow(weight, self.pow_p)  # [x,k_max,1]

        if len(inputs) == 3:
            k_user = tf.cast(tf.maximum(
                1.,
                tf.minimum(
                    tf.cast(self.k_max, dtype="float32"),  # k_max
                    tf.math.log1p(tf.cast(inputs[2], dtype="float32")) / tf.math.log(2.)  # hist_len
                )
            ), dtype="int64")
            
            seq_mask = tf.transpose(tf.sequence_mask(k_user, self.k_max), [0, 2, 1])
            padding = tf.ones_like(seq_mask, dtype=tf.float32) * (-2 ** 32 + 1)  # [x,k_max,1]
            weight = tf.where(seq_mask, weight, padding)

        weight = tf.nn.softmax(weight, name="weight")
        output = tf.reduce_sum(keys * weight, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (None, self.embedding_size)

    def get_config(self, ):
        config = {'k_max': self.k_max, 'pow_p': self.pow_p}
        base_config = super(LabelAwareAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [186]:
class CapsuleLayer(Layer):
    def __init__(self, input_units, out_units, max_len, k_max, iteration_times=3,
                 init_std=1.0, **kwargs):
        self.input_units = input_units
        self.out_units = out_units
        self.max_len = max_len
        self.k_max = k_max
        self.iteration_times = iteration_times
        self.init_std = init_std
        super(CapsuleLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.routing_logits = self.add_weight(shape=[1, self.k_max, self.max_len],
                                              initializer=RandomNormal(stddev=self.init_std),
                                              trainable=False, name="B", dtype=tf.float32)
        self.bilinear_mapping_matrix = self.add_weight(shape=[self.input_units, self.out_units],
                                                       initializer=RandomNormal(stddev=self.init_std),
                                                       name="S", dtype=tf.float32)
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        behavior_embddings, seq_len = inputs
        batch_size = tf.shape(behavior_embddings)[0]
        seq_len_tile = tf.tile(seq_len, [1, self.k_max])

        for i in range(self.iteration_times):
            mask = tf.sequence_mask(seq_len_tile, self.max_len)
            pad = tf.ones_like(mask, dtype=tf.float32) * (-2 ** 32 + 1)
            routing_logits_with_padding = tf.where(mask, tf.tile(self.routing_logits, [batch_size, 1, 1]), pad)
            weight = tf.nn.softmax(routing_logits_with_padding)
            behavior_embdding_mapping = tf.tensordot(behavior_embddings, self.bilinear_mapping_matrix, axes=1)
            Z = tf.matmul(weight, behavior_embdding_mapping)
            interest_capsules = squash(Z)
            delta_routing_logits = tf.reduce_sum(
                tf.matmul(interest_capsules, tf.transpose(behavior_embdding_mapping, perm=[0, 2, 1])),
                axis=0, keepdims=True
            )
            self.routing_logits.assign_add(delta_routing_logits)

        interest_capsules = tf.reshape(interest_capsules, [-1, self.k_max, self.out_units])
        return interest_capsules

    def compute_output_shape(self, input_shape):
        return (None, self.k_max, self.out_units)

    def get_config(self, ):
        config = {'input_units': self.input_units, 'out_units': self.out_units, 'max_len': self.max_len,
                  'k_max': self.k_max, 'iteration_times': self.iteration_times, "init_std": self.init_std}
        base_config = super(CapsuleLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    


def squash(inputs):
    vec_squared_norm = tf.reduce_sum(tf.square(inputs), axis=-1, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + 1e-8)
    vec_squashed = scalar_factor * inputs
    
    return vec_squashed

In [187]:
# create model

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, concatenate, Flatten, Dense, Dropout

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


def tile_user_otherfeat(user_other_feature, k_max):
        return tf.tile(tf.expand_dims(user_other_feature, -2), [1, k_max, 1])


def mind(
    sparse_input_length=1,
    dense_input_length=1,
    sparse_seq_input_length=50,
    
    embedding_dim = 64,
    neg_sample_num = 10,
    user_hidden_unit_list = [128, 64],
    k_max = 5,
    p = 1,
    dynamic_k = True
    ):
    

    
    # 1. Input layer
    user_id_input_layer = Input(shape=(sparse_input_length, ), name="user_id_input_layer")
    gender_input_layer = Input(shape=(sparse_input_length, ), name="gender_input_layer")
    age_input_layer = Input(shape=(sparse_input_length, ), name="age_input_layer")
    occupation_input_layer = Input(shape=(sparse_input_length, ), name="occupation_input_layer")
    zip_input_layer = Input(shape=(sparse_input_length, ), name="zip_input_layer")
    
    
    user_click_item_seq_input_layer = Input(shape=(sparse_seq_input_length, ), name="user_click_item_seq_input_layer")
    user_click_item_seq_length_input_layer = Input(shape=(sparse_input_length, ), name="user_click_item_seq_length_input_layer")
    
    
    pos_item_sample_input_layer = Input(shape=(sparse_input_length, ), name="pos_item_sample_input_layer")
    neg_item_sample_input_layer = Input(shape=(neg_sample_num, ), name="neg_item_sample_input_layer")


    
    # 2. Embedding layer
    user_id_embedding_layer = Embedding(6040+1, embedding_dim, mask_zero=True, name='user_id_embedding_layer')(user_id_input_layer)
    gender_embedding_layer = Embedding(2+1, embedding_dim, mask_zero=True, name='gender_embedding_layer')(gender_input_layer)
    age_embedding_layer = Embedding(7+1, embedding_dim, mask_zero=True, name='age_embedding_layer')(age_input_layer)
    occupation_embedding_layer = Embedding(21+1, embedding_dim, mask_zero=True, name='occupation_embedding_layer')(occupation_input_layer)
    zip_embedding_layer = Embedding(3439+1, embedding_dim, mask_zero=True, name='zip_embedding_layer')(zip_input_layer)
    
    item_id_embedding_layer = Embedding(3706+1, embedding_dim, mask_zero=True, name='item_id_embedding_layer')
    pos_item_sample_embedding_layer = item_id_embedding_layer(pos_item_sample_input_layer)
    neg_item_sample_embedding_layer = item_id_embedding_layer(neg_item_sample_input_layer)
    
    user_click_item_seq_embedding_layer = item_id_embedding_layer(user_click_item_seq_input_layer)

    

    
    ### ********** ###
    # 3. user part
    ### ********** ###
    
    # 3.1 pooling layer
    user_click_item_seq_embedding_layer_pooling = SequencePoolingLayer()([user_click_item_seq_embedding_layer, \
                                                                          user_click_item_seq_length_input_layer])
    
    print("user_click_item_seq_embedding_layer_pooling", user_click_item_seq_embedding_layer_pooling)
    
    
    # 3.2 capsule layer
    high_capsule = CapsuleLayer(input_units=embedding_dim,
                                out_units=embedding_dim, max_len=sparse_seq_input_length,
                                k_max=k_max)([user_click_item_seq_embedding_layer, user_click_item_seq_length_input_layer])
    
    print("high_capsule: ", high_capsule)
    

    # 3.3 Concat "sparse" embedding & "sparse_seq" embedding, and tile embedding
    other_user_embedding_layer = Flatten()(concatenate([user_id_embedding_layer, gender_embedding_layer, age_embedding_layer,
                                       occupation_embedding_layer, zip_embedding_layer, user_click_item_seq_embedding_layer_pooling], 
                                       axis=-1)
                                    )
    
    print("other_user_embedding_layer: ", other_user_embedding_layer)
    
    
    other_user_embedding_layer = tf.keras.layers.Lambda(tile_user_otherfeat, arguments={'k_max': k_max})(other_user_embedding_layer)
    print("other_user_embedding_layer: ", other_user_embedding_layer)
    
    
    
    # 3.4 user dnn part
    
    user_deep_input = concatenate([other_user_embedding_layer, high_capsule], axis=-1)
    print("user_deep_input: ", user_deep_input)

    
    for i, u in enumerate(user_hidden_unit_list):
        user_deep_input = Dense(u, activation="relu", name="FC_{0}".format(i+1))(user_deep_input)
        #user_deep_input = Dropout(0.3)(user_deep_input)
        
    print("user_deep_input: ", user_deep_input)
    
    dynamic_k = True
    if dynamic_k:
        user_embedding_final = LabelAwareAttention(k_max=k_max, pow_p=p, )(\
                                    [user_deep_input, pos_item_sample_embedding_layer, user_click_item_seq_length_input_layer])
    else:
        user_embedding_final = LabelAwareAttention(k_max=k_max, pow_p=p, )(\
                                    [user_deep_input, pos_item_sample_embedding_layer])
    
    
    user_embedding_final = tf.expand_dims(user_embedding_final, 1)
    print("user_embedding_final: ", user_embedding_final)
    
    
    
    ### ********** ###
    # 4. item part
    ### ********** ###

    item_embedding_layer = concatenate([pos_item_sample_embedding_layer, neg_item_sample_embedding_layer], \
                                       axis=1)
    
    item_embedding_layer = tf.transpose(item_embedding_layer, [0,2,1])
    
    print("item_embedding_layer: ", item_embedding_layer)

    ### ********** ###
    # 5. Output
    ### ********** ###
    dot_output = tf.matmul(user_embedding_final, item_embedding_layer)
    print(dot_output)
    dot_output = tf.nn.softmax(dot_output) # 输出11个值，index为0的值是正样本，负样本的索引位置为[1-10]
    
    print(dot_output)
    user_inputs_list = [user_id_input_layer, gender_input_layer, age_input_layer, \
                        occupation_input_layer, zip_input_layer, \
                        user_click_item_seq_input_layer, user_click_item_seq_length_input_layer]
    
    item_inputs_list = [pos_item_sample_input_layer, neg_item_sample_input_layer]

    model = Model(inputs = user_inputs_list + item_inputs_list,
                  outputs = dot_output)
    
    
    #print(model.summary())
    #tf.keras.utils.plot_model(model, to_file='MIND_model.png', show_shapes=True)


    model.__setattr__("user_input", user_inputs_list)
    model.__setattr__("user_embedding", user_deep_input)
    
    model.__setattr__("item_input", pos_item_sample_input_layer)
    model.__setattr__("item_embedding", pos_item_sample_embedding_layer)
    
    return model

In [188]:
#model = mind()

user_click_item_seq_embedding_layer_pooling Tensor("sequence_pooling_layer_50/Identity:0", shape=(None, 1, 64), dtype=float32)
high_capsule:  Tensor("capsule_layer_53/Identity:0", shape=(None, 5, 64), dtype=float32)
other_user_embedding_layer:  Tensor("flatten_44/Identity:0", shape=(None, 384), dtype=float32)
other_user_embedding_layer:  Tensor("lambda_43/Identity:0", shape=(None, 5, 384), dtype=float32)
user_deep_input:  Tensor("concatenate_119/Identity:0", shape=(None, 5, 448), dtype=float32)
user_deep_input:  Tensor("FC_2_41/Identity:0", shape=(None, 5, 64), dtype=float32)
user_embedding_final:  Tensor("ExpandDims_2:0", shape=(None, 1, 64), dtype=float32)
item_embedding_layer:  Tensor("transpose_29:0", shape=(None, 64, 11), dtype=float32)
Tensor("MatMul_24:0", shape=(None, 1, 11), dtype=float32)
Tensor("Softmax_24:0", shape=(None, 1, 11), dtype=float32)


In [164]:
# data generator

import numpy as np


def init_output():
    user_id = []
    gender = []
    age = []
    occupation = []
    zip = []
    hist_movie_id = []
    hist_len = []
    pos_movie_id = []
    neg_movie_id = []


    return user_id, gender, age, occupation, zip, \
        hist_movie_id, hist_len, pos_movie_id, neg_movie_id


def file_generator(input_path, batch_size):

    user_id, gender, age, occupation, zip, \
        hist_movie_id, hist_len, pos_movie_id, neg_movie_id = init_output()

    cnt = 0
    
    num_lines = sum([1 for line in open(input_path)])

    while True:

        with open(input_path, 'r') as f:
            for line in f.readlines():

                buf = line.strip().split('\t')

                user_id.append(int(buf[0]))
                gender.append(int(buf[1]))
                age.append(int(buf[2]))
                occupation.append(int(buf[3]))
                zip.append(int(buf[4]))
                hist_movie_id.append(np.array([int(i) for i in buf[5].strip().split(",")]))
                hist_len.append(int(buf[6]))
                pos_movie_id.append(int(buf[7]))
                neg_movie_id.append(np.array([int(i) for i in buf[8].strip().split(",")]))

                cnt += 1

                if cnt % batch_size == 0 or cnt == num_lines:
                    user_id = np.array(user_id, dtype='int32')
                    gender = np.array(gender, dtype='int32')
                    age = np.array(age, dtype='int32')
                    occupation = np.array(occupation, dtype='int32')
                    zip = np.array(zip, dtype='int32')
                    hist_movie_id = np.array(hist_movie_id, dtype='int32')
                    hist_len = np.array(hist_len, dtype='int32')
                    pos_movie_id = np.array(pos_movie_id, dtype='int32')
                    neg_movie_id = np.array(neg_movie_id, dtype='int32')

                    label = np.zeros(len(user_id)) # 正样本的index位置为0, 10个负样本的索引位置为[1-10]

                    yield [user_id, gender, age, occupation, zip, hist_movie_id, hist_len, pos_movie_id, neg_movie_id], label

                    user_id, gender, age, occupation, zip, hist_movie_id, hist_len, pos_movie_id, neg_movie_id = init_output()



In [165]:
# load data

train_path = "train.txt"
val_path = "test.txt"
batch_size = 1000

n_train = sum([1 for i in open(train_path)])
n_val = sum([1 for i in open(val_path)])

train_steps = n_train / batch_size
train_steps_ = n_train // batch_size
validation_steps = n_val / batch_size
validation_steps_ = n_val // batch_size


train_generator = file_generator(train_path, batch_size)
val_generator = file_generator(val_path, batch_size)

steps_per_epoch = train_steps_ if train_steps==train_steps_ else train_steps_ + 1
validation_steps = validation_steps_ if validation_steps==validation_steps_ else validation_steps_ + 1

print("n_train: ", n_train)
print("n_val: ", n_val)

print("steps_per_epoch: ", steps_per_epoch)
print("validation_steps: ", validation_steps)

n_train:  988129
n_val:  6040
steps_per_epoch:  989
validation_steps:  7


In [189]:
# train model

import os
import tensorflow as tf

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [early_stopping_cb]


model = mind()

model.compile(loss='sparse_categorical_crossentropy', \
    optimizer=Adam(lr=1e-3), \
    metrics=['sparse_categorical_accuracy'])
# loss="sparse_categorical_accuracy"的应用方式参见：https://mp.weixin.qq.com/s/H4ET0bO_xPm8TNqltMt3Fg



history = model.fit(train_generator, \
                    epochs=2, \
                    steps_per_epoch = steps_per_epoch, \
                    callbacks = callbacks, 
                    validation_data = val_generator, \
                    validation_steps = validation_steps, \
                    shuffle=True
                   )

user_click_item_seq_embedding_layer_pooling Tensor("sequence_pooling_layer_51/Identity:0", shape=(None, 1, 64), dtype=float32)
high_capsule:  Tensor("capsule_layer_54/Identity:0", shape=(None, 5, 64), dtype=float32)
other_user_embedding_layer:  Tensor("flatten_45/Identity:0", shape=(None, 384), dtype=float32)
other_user_embedding_layer:  Tensor("lambda_44/Identity:0", shape=(None, 5, 384), dtype=float32)
user_deep_input:  Tensor("concatenate_122/Identity:0", shape=(None, 5, 448), dtype=float32)
user_deep_input:  Tensor("FC_2_42/Identity:0", shape=(None, 5, 64), dtype=float32)
user_embedding_final:  Tensor("ExpandDims_3:0", shape=(None, 1, 64), dtype=float32)
item_embedding_layer:  Tensor("transpose_30:0", shape=(None, 64, 11), dtype=float32)
Tensor("MatMul_25:0", shape=(None, 1, 11), dtype=float32)
Tensor("Softmax_25:0", shape=(None, 1, 11), dtype=float32)
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 989 steps, validate for 7 steps
Epoch 1/2
989/989 [====================

In [190]:
model.save_weights('mind_model.h5')


In [192]:
re_model = mind()
re_model.load_weights('mind_model.h5')

print(re_model.summary())

user_click_item_seq_embedding_layer_pooling Tensor("sequence_pooling_layer_52/Identity:0", shape=(None, 1, 64), dtype=float32)
high_capsule:  Tensor("capsule_layer_55/Identity:0", shape=(None, 5, 64), dtype=float32)
other_user_embedding_layer:  Tensor("flatten_46/Identity:0", shape=(None, 384), dtype=float32)
other_user_embedding_layer:  Tensor("lambda_45/Identity:0", shape=(None, 5, 384), dtype=float32)
user_deep_input:  Tensor("concatenate_125/Identity:0", shape=(None, 5, 448), dtype=float32)
user_deep_input:  Tensor("FC_2_43/Identity:0", shape=(None, 5, 64), dtype=float32)
user_embedding_final:  Tensor("ExpandDims_4:0", shape=(None, 1, 64), dtype=float32)
item_embedding_layer:  Tensor("transpose_31:0", shape=(None, 64, 11), dtype=float32)
Tensor("MatMul_26:0", shape=(None, 1, 11), dtype=float32)
Tensor("Softmax_26:0", shape=(None, 1, 11), dtype=float32)
Model: "model_26"
__________________________________________________________________________________________________
Layer (type)  

In [193]:
user_id, gender, age, occupation, zip, \
        hist_movie_id, hist_len, pos_movie_id, neg_movie_id = init_output()

with open("test.txt", 'r') as f:
    for line in f.readlines():

        buf = line.strip().split('\t')

        user_id.append(int(buf[0]))
        gender.append(int(buf[1]))
        age.append(int(buf[2]))
        occupation.append(int(buf[3]))
        zip.append(int(buf[4]))
        hist_movie_id.append(np.array([int(i) for i in buf[5].strip().split(",")]))
        hist_len.append(int(buf[6]))
        pos_movie_id.append(int(buf[7]))
        

user_id = np.array(user_id, dtype='int32')
gender = np.array(gender, dtype='int32')
age = np.array(age, dtype='int32')
occupation = np.array(occupation, dtype='int32')
zip = np.array(zip, dtype='int32')
hist_movie_id = np.array(hist_movie_id, dtype='int32')
hist_len = np.array(hist_len, dtype='int32')
pos_movie_id = np.array(pos_movie_id, dtype='int32')


In [194]:
# Generate user features for testing and full item features for retrieval

test_user_model_input = [user_id, gender, age, occupation, zip, hist_movie_id, hist_len]
all_item_model_input = list(range(0, 3706+1))

user_embedding_model = Model(inputs=re_model.user_input, outputs=re_model.user_embedding)
item_embedding_model = Model(inputs=re_model.item_input, outputs=re_model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)


user_embs = np.reshape(user_embs, (-1, 64))
item_embs = np.reshape(item_embs, (-1, 64))

print(user_embs[:2])
print(item_embs.shape)

(6040, 5, 64)
(3707, 1, 64)
[[0.         0.14731093 0.13900717 0.         0.         1.478808
  0.         0.18150645 0.80898654 0.         0.         0.
  0.         0.         0.         0.15728728 0.08639187 0.42935494
  0.7397017  0.06567826 0.41014    0.         0.16195668 1.5011536
  0.         0.         0.         0.05010591 0.         0.19825594
  0.         0.         0.9191648  0.7207551  0.67062324 0.6644707
  0.47457296 0.13724752 0.26177412 0.13435438 0.10145954 0.
  0.28838137 0.57013756 0.         0.         0.         0.
  0.1595294  0.7744232  0.04367758 0.52349055 0.63513637 0.
  0.         0.09334061 0.0245292  0.         0.8633623  0.
  0.         0.42910543 0.         1.0345823 ]
 [0.         0.         0.24033175 0.         0.         1.5294596
  0.         0.0761693  0.90816593 0.         0.06147628 0.1498693
  0.         0.         0.         0.22493754 0.         0.46198222
  0.66455394 0.         0.4079227  0.         0.16429693 1.5069313
  0.         0.     

In [195]:
# Generate user features for testing and full item features for retrieval

test_user_model_input = [user_id, gender, age, occupation, zip, hist_movie_id, hist_len]
all_item_model_input = list(range(0, 3706+1))

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)


user_embs = np.reshape(user_embs, (-1, 64))
item_embs = np.reshape(item_embs, (-1, 64))

print(user_embs[:2])
print(item_embs.shape)

(6040, 5, 64)
(3707, 1, 64)
[[0.         0.14731093 0.13900717 0.         0.         1.478808
  0.         0.18150645 0.80898654 0.         0.         0.
  0.         0.         0.         0.15728728 0.08639187 0.42935494
  0.7397017  0.06567826 0.41014    0.         0.16195668 1.5011536
  0.         0.         0.         0.05010591 0.         0.19825594
  0.         0.         0.9191648  0.7207551  0.67062324 0.6644707
  0.47457296 0.13724752 0.26177412 0.13435438 0.10145954 0.
  0.28838137 0.57013756 0.         0.         0.         0.
  0.1595294  0.7744232  0.04367758 0.52349055 0.63513637 0.
  0.         0.09334061 0.0245292  0.         0.8633623  0.
  0.         0.42910543 0.         1.0345823 ]
 [0.         0.         0.24033175 0.         0.         1.5294596
  0.         0.0761693  0.90816593 0.         0.06147628 0.1498693
  0.         0.         0.         0.22493754 0.         0.46198222
  0.66455394 0.         0.4079227  0.         0.16429693 1.5069313
  0.         0.     

In [44]:
import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

embedding_dim = 64
index = faiss.IndexFlatIP(embedding_dim)
index.add(item_embs)

D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0

for i, uid in tqdm(enumerate(user_id)):
    try:
        pred = [all_item_model_input[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(pos_movie_id[i], pred, N=50)
        s.append(recall_score)
        if pos_movie_id[i] in pred:
            hit += 1
    except:
        print(i)
        
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(user_id))

ModuleNotFoundError: No module named '_swigfaiss'